* https://github.com/derekgreene/topic-model-tutorial 
* https://github.com/derekgreene/topic-model-tutorial/blob/master/topic-modelling-with-scikitlearn.pdf
* https://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html

In [1]:
#######################
#basic standard modules
#######################
import sys, os
import time
import collections, itertools, copy, operator

#######################
#custom config modules
#######################
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')
load_dotenv()

#######################
#file manipulation modules
#######################
import pickle, json

#######################
#string manipulation modules
#######################
import re, string
import nltk
print("in the other",os.getenv("TEST_MULTILINE_VAR"))
nltk.data.path.append(os.getenv("NLTKDATADIR"))
from bs4 import BeautifulSoup

#######################
#general data manipulation and data analysis modules
#######################
import pandas, gensim, sklearn, scipy, numpy, math

#######################
#custom modules
#######################
import processingData

in the other hello\nworld
in the other hello\nworld


In [2]:
############################
## extract_candidate_chunks: candidate phrases based on http://bdewilde.github.io/blog/2014/09/23/intro-to-automatic-keyphrase-extraction/
############################
def extract_candidate_chunks(lemmpostxt, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}', STOPWORDS = nltk.corpus.stopwords.words('english')):
    #def redotaggedtext(lemmpostxt, lemmpossts = [], lemmposst = []):
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    lemmpostxt_redo = []
    for lemmposst in lemmpostxt:
        lemmposst_redo = []
        for w,val,l,pos in lemmposst:
            if w not in STOPWORDS:
                #or not all(char in punct for char in w):
                lemmposst_redo.append((l,pos))
        lemmpostxt_redo.append(lemmposst_redo)
            
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(lemmposst_redo)) for lemmposst_redo in lemmpostxt_redo))

    #print(all_chunks)
    # join constituent chunk words into a single chunked phrase
    def createcands(group):
        total = []
        current = ''
        #print(type(list(group))) #group is an ITERATOR; once I call it, it will dissappear unless I do something else
        #print(list(group)[::-1])
        reversing = [None]
        #for el in group:
        #    print(el)
        for el in group:
            if reversing[0] == None:
                reversing[0] = el[0]
            else:
                reversing.insert(0, el[0])
        #print(reversing)
        for word in reversing:
            if current == '':
                current = word
            else:
                current = word + ' ' + current
            total.append(current)
        #print(list(set(reversing+total)))
        return list(set(reversing+total))
    
    def createcands2(group):
        total = []
        current = ''
        #print(type(list(group))) #group is an ITERATOR; once I call it, it will dissappear unless I do something else
        for word, pos, chunk in list(group):
            if len(word) < 2:
                continue
            #print(word)
            if current == '':
                current = word
                total.append(word)
            else:
                current = current+' '+word
                total.append(word)
        total.append(current)

        return list(set(total))
            
    #candidates = [' '.join(word for word, pos, chunk in group)
    candidates = [createcands2(group)
                    for key, group in itertools.groupby(all_chunks, lambda w_TUPLE: w_TUPLE[2] != 'O') if key]
    #print(candidates)

    #return [cand for cand in candidates]
    #print(list(itertools.chain.from_iterable(candidates)))
    return list(itertools.chain.from_iterable(candidates))

In [50]:
############################
## allrecordsPreparation 3: revision of 2 to extend it for keyphrase candidate analysis
## some articles:
## -- https://stackoverflow.com/questions/34714162/preventing-splitting-at-apostrophies-when-tokenizing-words-using-nltk
############################

def allrecordsPreparation3(allrecords, STOPWORDS=nltk.corpus.stopwords.words('english'), punct = set(string.punctuation)):
    '''
    description: tokenization and POS tagging
    input: dict of allrecords texts and data from different sources
    treatment: separating only those with posts in the forum and tokenizing the posts
    output:
    1) list of lists, each with:
    -- id
    -- username
    -- link of the post
    -- tokenized text
    -- POS tagged text
    2) list of post ids
    '''
    print('in allrecordsPreparation (len(allrecords))::',len(allrecords))
 
       
    def HTMLtoText(u):
        #block arguments
        record = u["data"]
        forumpost = record['forum']['foundjob_msg']['text']
        if forumpost == '':
            return False
        forumpostID = record['forum']['foundjob_msg']['id']
        forumpostLINK = record['forum']['foundjob_msg']['link']
        soup_forumpost = BeautifulSoup(forumpost)
        pattern_A01 = re.compile(r'^hey( |$)')
        
        ## Text extraction, REGEXs and Replacements
        soup_forumpostTEXT = soup_forumpost.find('body').get_text() \
                                                        .lower() \
                                                        .replace('’',"'") \
                                                        .replace('‘', "'") \
                                                        .replace('´', "'") \
                                                        .replace('free code camp', 'fcc') \
                                                        .replace('javascript', 'js') \
                                                        .replace('part time', 'parttime') \
                                                        .replace('part-time', 'parttime') \
                                                        .replace('full time', 'fulltime') \
                                                        .replace('full-time', 'fulltime') \
                                                        .replace('web application', 'web app') \
                                                        .replace('web development', 'web dev') \
                                                        .replace('developer', 'dev') \
                                                        .replace('front end', 'frontend') \
                                                        .replace('front-end','frontend') \
                                                        .replace('back end', 'backend') \
                                                        .replace('back-end','backend') \
                                                        .replace('full stack', 'fullstack') \
                                                        .replace('full-stack','fullstack') \
                                                        .replace('resume', 'cv') \
                                                        .replace('hello', 'hi') \
                                                        .replace(' angular ', ' angularjs ')
        
        soup_forumpostTEXT = re.sub(pattern_A01, ' hello ', soup_forumpostTEXT)
        
        tksoup_forumpostTEXT = [
                                #nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) 
                                token
                                for token in nltk.word_tokenize(soup_forumpostTEXT)
                                #if token.lower() not in STOPWORDS 
                                #and not re.match(r'\d+?', token)
                                #and not all(char in set(string.punctuation) for char in token)
                               ]
        return forumpostID, forumpostLINK, tksoup_forumpostTEXT
    
    def approxsts(text):
        modtext = []
        for w in tksoup_forumpostTEXT:
            w = w.lower()
            rws = []
            if len(w) > 1 and len({'.','-',':'}.intersection(w)) >= 1:
                #print(w)
                for punc in {'.','-',':'}.intersection(w):
                    rws = w.replace(punc, ' '+punc+' ').split()
                #print(rws)
            if len(rws) == 0:
                modtext.append(w)
            else:
                for w in rws:
                    modtext.append(w)
        return modtext
    
    def lemmatizationofpos(postxt, lemmws):
        lemmposws = []
        counterrors = 0
        countKerrors = 0
        countIerrors = 0
        countNCerrors = 0
        for posw in postxt:
            w = posw[0]
            pos = posw[1]
            if nltk.corpus.wordnet.synsets(w):
                try:
                    n = ''
                    if nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0] == 'entity':
                        n = w
                    else:
                        n = nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0]
                    if pos[0] == 'V':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w,'v')
                    if pos == 'NNS' or pos == 'NN$':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w)
                    #print(w, nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms(), n)
                    lemmposws.append((w,True,n,pos))
                    lemmws.append(n)
                except KeyError: #in some cases the POS tag is not recognised by wordnet synset
                    #print("pos KeyErrors", w,pos)
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
                    counterrors += 1
                    countKerrors += 1
                except IndexError:
                    #print("IndexErrors (POS not found)", w,pos) #in some cases (w,pos) pair was not found at some point of the synsets root hyernyms
                    #print(nltk.corpus.wordnet.synsets(w,pos[0].lower()))
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
                    counterrors += 1
                    countIerrors += 1
            else:
                #print(w, [])
                #print("Error (not in corpus)", w,pos)
                lemmposws.append((w,False,w,pos))
                lemmws.append(w)
                counterrors += 1
                countNCerrors += 1
        #print("\n\ntotal number of errors : ", counterrors)
        #print("total number of noPOSerrors : ", countKerrors)
        #print("total number of indexerrors : ", countIerrors)
        #print("total number of nonincorpuserrors : ", countNCerrors)
        return lemmposws, counterrors
      
    
    lemmws = []
    lemmposrecs = []
    count = 0
    lemerrors = 0
    for u in allrecords:
        ## Getting the data as a text from HTML format (raw dataset)
        userdata = HTMLtoText(u)
        if userdata != False:
            forumpostID, forumpostLINK, tksoup_forumpostTEXT = userdata
            
            ## Sentence identification, tokenization and POS
            txt2possts = []
            st = []
            for token in tksoup_forumpostTEXT:
                st.append(token)
                if re.match(r'^[.!?]+|\n$', token):
                    postst = nltk.pos_tag(st)
                    txt2possts.append(postst)
                    st = []
            if not re.match(r'^[.!?]+|\n$', token):
                posst = nltk.pos_tag(st)
                txt2possts.append(posst)

           
            ## Lemm text
            lemmpostxt = []
            for posst in txt2possts:
                lemst, err = lemmatizationofpos(posst, lemmws)
                lemmpostxt.append(lemst)
                lemerrors += err

            
            ## Keyphrases candidates; complete lemmws with candidates that are not still there
            candidates = extract_candidate_chunks(lemmpostxt)
            for cand in candidates:
                if cand not in lemmws:
                    lemmws.append(cand)
            
            ## Adding data to the new created dataset
            lemmposrecs.append((
                            'f_'+forumpostID,
                            u["user"],
                            forumpostLINK,
                            lemmpostxt,
                            candidates
                            ))
            count += 1

    print("number of treated posts (len(count)) ::", count)
    print("lemm errs:", lemerrors)
    #return all_posedsts, forum_ids
    return lemmposrecs, nltk.FreqDist(lemmws)

In [13]:
############################
## wordimportance_var4: wordimportance_var3 modified to fit keyphrases; changes in the "opacity" formula
############################
def wordimportance_var4(lemmposrecs, lemmws_fd):
    '''
    description:
    
    This metric tries to solve some of the issues that appeared in `wordimportance_var1` metric, in particular the values of zero.
    
    This is done by setting lower bounds when required.
    
    --- In the case of opacity, a non-zero lower bound is set by changing the equation to the following:
        ```
        if 1-math.log(v)/maxdiv == 0: 1-math.log(maxdiv-1)/maxdiv # 1-math.log(v)/maxdiv == 0 if v == maxdiv
        ```
    --- In the case of sizing, a redefinition of the metric force a non-zero lower bound as well as rebumpimg rare terms in documents:
        ```
        (sum(vector)-max(vector))/sum(vector)
        ```
    
    input:
        1) tokenized list of texts
        2) freqDist of lemmatized words
    
    output: wordimportance 
    '''
    unigrams = dict([(grams, count) 
                     for grams, count in lemmws_fd.items() 
                     if len(grams.split()) == 1 or (len(grams.split()) == 2 and '' in grams.split())
                    ])
    maxdiv = math.log(sorted(unigrams.items(), key=lambda x: x[1], reverse=True)[0][1])
    opacity = collections.defaultdict(float)
    for grams, counts in lemmws_fd.items():
        opval = []
        for gram in grams.split():
            if gram == '':
                continue
            if gram in unigrams:
                opval.append(math.log(unigrams[gram]))
            else:
                opval.append(0)
        averopval = sum(opval)/len(opval)
        if 1 - averopval/maxdiv != 0.0:
            opacity[grams] = 1 - averopval/maxdiv
        else:
            opacity[grams] = 1-math.log(maxdiv-1)/maxdiv
        
    sizing_matrix = dict([(w, [0]*len(lemmposrecs)) for w in list(lemmws_fd.keys())])

    ## Count lemmatized words/characters per text  
    for i,lemmpos_t in enumerate(lemmposrecs):
        for k, lemmpos_TUPLE in enumerate(lemmpos_t[3]):
            ## Use lemmatized word
            lemmw = lemmpos_TUPLE[2]
            sizing_matrix[lemmw][i] = sizing_matrix[lemmw][i] + 1
        for cand in lemmpos_t[4]: #<--------------------------------- problem!! it counted candidates only once!!!
            if cand not in sizing_matrix:
                sizing_matrix[cand][i] = sizing_matrix[cand][i] + 1

       
    ## Normalization
    normalization = dict([(k, (sum(vector)-max(vector))/sum(vector)) if sum(vector) != 0 else (k,0.0) for k, vector in sizing_matrix.items()])
    #normalization = collections.defaultdict(float)
    #for k, vector in sizing_matrix.items():
    #    if sum(vector) != 0:
    #        normalization[k] = (sum(vector)-max(vector))/sum(vector)
    #    else:
    #        
        
    
    wordimportance = dict([(k, valnorm*opacity[k]) for k, valnorm in normalization.items()])

    return wordimportance

In [14]:
def cleaningtext(st, STOPWORDS = nltk.corpus.stopwords.words('english')):
    treated_st = []
    countwds = len(st)
    for w in st:
        if 'freecodecamp' in w:
            w = w.replace('freecodecamp','fcc')
        treated_st.append(w)
    return treated_st, countwds

In [56]:
############################
## gensim_models2 re-evaluated: modification of gensim_model2's cleanedsts_from_lemmpostxts to fit keyphrases
############################

def gensim_models2(lemmposrecs, NUM_TOPICS = 15, lemmws_fd = {}, wordimportance = {}, nltk = nltk, gensim = gensim):
    
    def cleanedsts_from_lemmpostxts2(lemmposrecs, STOPWORDS = nltk.corpus.stopwords.words('english')):
        redo_corpus_by_sts = []
        for lemmpos_r in lemmposrecs:
            candidates = lemmpos_r[4]
            treated_lemmcands = cleaningtext(candidates)[0] #passing candidates only
            redo_corpus_by_sts.append(treated_lemmcands) 
        
        #print(len(redo_corpus_by_sts), redo_corpus_by_sts[-1])
        return redo_corpus_by_sts
        
    def basedonBOW(redo_corpus_by_sts):
        dictionary = gensim.corpora.Dictionary(redo_corpus_by_sts) #[token for st in redo_corpus_by_sts for token in st]
        corpus = [dictionary.doc2bow(text) for text in redo_corpus_by_sts]
        return corpus, dictionary
    
    def basedonTFIDF(corpus):
        return gensim.models.TfidfModel(corpus)
    
    def basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance):
        
        def metriccalc(w):
            if w in wordimportance:
                return 1.0+2.0**float(wordimportance[w])
            else:
                return 0.0

        corpus = []
        for sts in redo_corpus_by_sts:
            st = []
            for w in sts:
                st.append((dictionary.token2id[w], metriccalc(w)))
            corpus.append(st)
        return corpus
        
    
    redo_corpus_by_sts  = cleanedsts_from_lemmpostxts2(lemmposrecs)
    
    corpus, dictionary = basedonBOW(redo_corpus_by_sts)
    if wordimportance == {'tfidf':True}:
        tfidf = basedonTFIDF(corpus)
        corpus = tfidf[corpus]
    if wordimportance != {} and wordimportance != {'tfidf':True}:
        corpus = basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance)
    
    
    lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, minimum_probability=0.005, iterations=100, per_word_topics = True, minimum_phi_value = 0.001, id2word=dictionary)
    lsi_model = gensim.models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

   
    return lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary

In [16]:
with open('../data/jobproject_forum.json','r') as message:
    otp = json.load(message)
print(len(otp))
allrecords = [{ "user": k, "data": otp[k] }  for k in otp]
print(len(allrecords))

90
90


In [51]:
lemmposrecs, lemmws_fd = allrecordsPreparation3(allrecords)

in allrecordsPreparation (len(allrecords)):: 90


/home/ec/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


number of treated posts (len(count)) :: 53
lemm errs: 19296


In [18]:
#lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, 
#                                                                                NUM_TOPICS=20,
#                                                                                wordimportance = {'tfidf':True})

#lda_model.print_topics(num_words=15)

#[' '.join([l for wr in rec[3] for w,_,l,pos in wr]) for rec in lemmposrecs]
#[[l for wr in rec[3] for w,_,l,pos in wr] for rec in lemmposrecs]
#[' '.join([cand for cand in rec[4]]) for rec in lemmposrecs]
##https://stackoverflow.com/questions/46282473/error-while-identify-the-coherence-value-from-lda-model
#texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus]
#texts

#coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
#                                                                  texts=texts,
#                                                                  #corpus=corpus,
#                                                                  window_size=20,
#                                                                  dictionary=dictionary, 
#                                                                  coherence='c_uci')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

#coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, 
#                                                   texts=texts, 
#                                                   dictionary=dictionary,
#                                                   coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

## Compute Coherence Score using UMass
#coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
#                                     #texts=[[l for wr in rec[3] for w,_,l,pos in wr] for rec in lemmposrecs],
#                                     corpus = corpus,
#                                     dictionary=dictionary, 
#                                     coherence="u_mass")
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

#print('\nPerplexity: ', lda_model.log_perplexity(corpus))

#dictionary
#corpus

TOPICS = range(2,61,2)
cv_coherence_values = []
for numtopics in TOPICS:
    print('\n\nFor NUM_TOPICS:', numtopics)
    lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=numtopics, wordimportance = {'tfidf':True})
    umass_coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
                                     #texts=[[l for wr in rec[3] for w,_,l,pos in wr] for rec in lemmposrecs],
                                     corpus = corpus,
                                     dictionary=dictionary, 
                                     coherence="u_mass")
    umass_coherence_lda = umass_coherence_model_lda.get_coherence()
    print('U-Mass Coherence Score: ', umass_coherence_lda)
    texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus]
    cv_coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
                                     texts=texts,
                                     dictionary=dictionary, 
                                     coherence="c_v")
    cv_coherence_lda = cv_coherence_model_lda.get_coherence()
    cv_coherence_values.append(cv_coherence_lda)
    print('C_V Coherence Score: ', cv_coherence_lda)
    print('Perplexity: ', lda_model.log_perplexity(corpus))

In [19]:
#https://www.quora.com/Can-I-combine-LSI-and-K-means-for-text-document-clustering-Are-there-any-sources-to-learn-about-it
#http://sujitpal.blogspot.com/2014/08/topic-modeling-with-gensim-over-past.html

##https://stackoverflow.com/questions/14261903/how-can-i-open-the-interactive-matplotlib-window-in-ipython-notebook
%matplotlib inline 
import matplotlib.pyplot as plt
limit=61; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, cv_coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [20]:
#lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=40, wordimportance = {'tfidf':True})

In [21]:
#lda_model.print_topics(num_topics=40)

In [57]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=20, wordimportance = {'tfidf':True})

In [53]:
lsi_model.print_topics(num_topics=30, num_words=15)

[(0,
  '0.124*"project" + 0.100*"company" + 0.097*"interview" + 0.097*"people" + 0.094*"lot" + 0.093*"code" + 0.092*"way" + 0.092*"app" + 0.086*"cv" + 0.086*"skill" + 0.085*"day" + 0.084*"time" + 0.079*"experience" + 0.078*"web" + 0.078*"recruiter"'),
 (1,
  '-0.226*"lawyer" + -0.224*"fellow camper" + -0.224*"fellow" + -0.168*"success" + -0.126*"story" + -0.121*"camper" + -0.113*"ideas.ataccama.com" + -0.113*"satisfied" + -0.113*"dev fcc" + -0.113*"satisfied current job" + -0.107*"internship" + -0.105*"medium article" + -0.101*"proud" + -0.101*"friend local fcc group everyone" + -0.101*"success story fcc forum"'),
 (2,
  '-0.174*"design" + -0.124*"graphic" + -0.114*"internship" + 0.093*"github" + 0.092*"manager" + 0.088*"app" + 0.088*"recruiter" + 0.079*"source" + 0.076*"open" + -0.074*"lawyer" + -0.071*"agency" + 0.070*"city" + -0.070*"role" + -0.069*"hi" + 0.068*"company"'),
 (3,
  '-0.113*"taught" + -0.104*"app" + -0.102*"css js" + -0.101*"study" + -0.100*"adwords" + -0.100*"logic" 

In [54]:
#import pyLDAvis
#import pyLDAvis.gensim
#pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

In [58]:
lda_model.print_topics(num_topics=20, num_words=20)

[(0,
  '0.001*"machine" + 0.001*"much time learning" + 0.001*"month graduation" + 0.001*"local code camp armenia" + 0.001*"armenia" + 0.001*"bright day" + 0.001*"codacademy" + 0.001*"bright" + 0.001*"political science" + 0.001*"web dev offer" + 0.001*"good kind resource" + 0.001*"science" + 0.001*"graduation" + 0.001*"political" + 0.001*"new job" + 0.001*"pay" + 0.001*"camp" + 0.001*"profession" + 0.001*"backend" + 0.001*"everyone"'),
 (1,
  '0.002*"github" + 0.002*"thats" + 0.001*"everyday" + 0.001*"frontend dev" + 0.001*"july" + 0.001*"certificate" + 0.001*"challenge" + 0.001*"gulp" + 0.001*"day" + 0.001*"react" + 0.001*"right" + 0.001*"accomplishment" + 0.001*"source" + 0.001*"blog" + 0.001*"open" + 0.001*"//manuelbasanta.github.io/" + 0.001*"cs50" + 0.001*"awsome course" + 0.001*"data course" + 0.001*"course amazing it´s inttoductory"'),
 (2,
  '0.002*"design" + 0.001*"graphic" + 0.001*"sleep" + 0.001*"friday" + 0.001*"mind" + 0.001*"site" + 0.001*"student" + 0.001*"meetup" + 0.001

In [ ]:
lda_model.get_topic_terms(0)

In [ ]:
dictionary.id2token[2298]

In [ ]:
#https://radimrehurek.com/gensim/models/ldamodel.html
#https://radimrehurek.com/gensim/corpora/dictionary.html
for t in range(10):
    print('\n\nTopic num.', str(t),' :')
    for ids, prob in lda_model.get_topic_terms(t):
        print(dictionary.id2token[ids])

### TODO
k-means to approximate the number of topics before trying a more elaborate form
Check and improve previous work:
* https://github.com/evaristoc/fccgitterDataScience/blob/master/Identifying%20Relevant%20Topics%20in%20a%20Chatroom.ipynb
* https://stackoverflow.com/questions/24816912/number-of-latent-semantic-indexing-topics
* https://stackoverflow.com/questions/9582291/how-do-we-decide-the-number-of-dimensions-for-latent-semantic-analysis/9759218#9759218

Also check:
* https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/ (good example but conceptually a bit wrong)
* https://radimrehurek.com/gensim/models/hdpmodel.html
* http://hojunhao.github.io/sgparliament/LDA.html
* https://datascience.stackexchange.com/questions/128/latent-dirichlet-allocation-vs-hierarchical-dirichlet-process
* https://nlpforhackers.io/recipe-text-clustering/
* https://www.quora.com/Latent-Dirichlet-Allocation-LDA-What-is-the-best-way-to-determine-k-number-of-topics-in-topic-modeling
* http://nbviewer.jupyter.org/github/bmabey/hacker_news_topic_modelling/blob/master/HN%20Topic%20Model%20Talk.ipynb#topic=3&lambda=1&term=
* https://stackoverflow.com/questions/50106516/k-means-for-topic-modelling-elbow-method
* https://shuaiw.github.io/2016/12/22/topic-modeling-and-tsne-visualzation.html
* http://wdsinet.org/Annual_Meetings/2016_Proceedings/papers/Paper45.pdf
* http://ramet.elte.hu/~podani/Methods.htm
* https://hk.saowen.com/a/edc29232eae094158f66e8ff3f08d6f35b8a2a45d628fce8917d2dce6f94282e
* https://rare-technologies.com/validating-gensims-topic-coherence-pipeline/
* http://sujitpal.blogspot.com/2014/08/topic-modeling-with-gensim-over-past.html
* https://www.searchenginejournal.com/latent-semantic-indexing-wont-help-seo/240705/
* https://www.quora.com/What-is-topic-coherence + http://qpleple.com/topic-coherence-to-evaluate-topic-models/
* https://stackoverflow.com/questions/50340657/pyldavis-with-mallet-lda-implementation-ldamallet-object-has-no-attribute-inf
* https://markroxor.github.io/gensim/static/notebooks/gensim_news_classification.html

In [ ]:
END HERE

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans

NUM_CLUSTERS = 8

from scipy.spatial.distance import cdist, pdist
K = list(range(1, n_components+1))
KM = [KMeans(n_clusters=k).fit(X) for k in K]
centroids = [k.cluster_centers_ for k in KM]

D_k = [cdist(X, cent, 'euclidean') for cent in centroids]
cIdx = [numpy.argmin(D,axis=1) for D in D_k]
dist = [numpy.min(D,axis=1) for D in D_k]
avgWithinSS = [sum(d)/X.shape[0] for d in dist]


kIdx = 8-1

# elbow curve
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, avgWithinSS, 'b*-')
ax.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12, 
markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elbow for KMeans clustering')


###############################################################################
# Do the actual clustering

true_k = 8


km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

print("Clustering sparse data with %s" % km)
newX = numpy.array(pandas.concat([pandas.DataFrame(X),datadf_foran['timestamp_norm'].reset_index()['timestamp_norm']],axis=1))
km.fit(newX)

print()

labels = [x for x in range(datadf_foran.shape[0])]
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(newX, km.labels_, sample_size=1000))

print()


fig_clusters = plt.figure()
fig_clusters.suptitle('Clusters over first 2 Components')
ax = fig_clusters.add_subplot(111)
ax.set_xlabel('Component I')
ax.set_ylabel('Component II')
plt.scatter(newX[:,0],newX[:,1], c=km.fit_predict(newX))
plt.show()

### TODO

In [ ]:
############################
## raw_lda_frankjupyter2: modified to fit allrecordsPreparation2
############################
def raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=2, STOPWORDS=nltk.corpus.stopwords.words('english')):
    '''
    description: modified model based on https://www.frankcleary.com/svd/ for a more raw construction of a lda
    '''
    
   
    def metriccalc(st, normalizer, wordimportance):
        '''
        description:
        text normalization based on ALL characters in the sentence; why? Example: if two writers wrote 20 words, 2 of them very important, but one of them wrote half of characters stopwords, those 2 words wouldnt be penalized accordingly for this writer: the other wrote more important content
        '''       
        metfuncs = [
            lambda w: math.pow(0.1+float(wordimportance[w]),textbow[w]/normalizer) if w in list(wordimportance.keys()) else 0.0, #a sort of idf-normalization based on number of words in the text: the more the words in a text, the more important
            lambda w: float(wordimportance[w])*textbow[w] if w in list(wordimportance.keys()) else 0.0, #good but ignore those words with worimportance too low or 0 but that are frequent in text
            lambda w: 1.0+2.0**float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0, #<- probably the best one; because it is not normilized this indicator would simply say that if it has the word at least once is already on topic
            lambda w: float(wordimportance[w]) + textbow[w]/normalizer if w in list(wordimportance.keys()) else 0.0,
            lambda w: (1.0+textbow[w]/normalizer)*float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0, #<- apparently the second best with the adhoc word ranking
            lambda w: normalizer/textbow[w]*wordimportance[w]/sum(list(wordimportance.values())), #tfidf-ish
            lambda w: float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0
        ]
        likedict = collections.defaultdict(float)
        textbow = collections.Counter(st)
        for w in st:
            likedict[w] = metfuncs[metmodel](w)
        return likedict

    #redo_corpus_by_sts = []
    words_df = pandas.DataFrame()
    textreference = {}
    
    
    for textindex, lemmpos_r in enumerate(lemmposrecs):
        lemmpos_t = lemmpos_r[3]
        #print('lemmpos_t', len(lemmpos_t))
        lemm_sts = ''
        for lemmpos_TUPLE in lemmpos_t:
            w = lemmpos_TUPLE[2]
            lemm_sts = lemm_sts + w + ' '
        sts = lemm_sts.split('.')
        for stindex, lemmpos_st in enumerate(sts):
            treated_st, lensts = cleaningtext(lemmpos_st.split(), STOPWORDS=STOPWORDS)        
            #print('treated_st', lensts)
            if len(treated_st) > 3:
                likedict = metriccalc(treated_st, lensts, wordimportance)
                st_df = pandas.DataFrame.from_dict(likedict, orient='index')
                textindexing = str(textindex)+'_'+str(stindex)
                st_df.columns = [textindexing]
                textreference[textindexing] = {}
                textreference[textindexing]['treated_st'] = treated_st
                #st_df.columns = [str(count)]
                words_df = words_df.join(st_df, how='outer', )
    
    words_df = words_df.fillna(0)
    print("Number of unique words: %s" % len(words_df))
    print(words_df.head(10))
    #print(words_df.sort(columns=words_df.columns[0], ascending=False).head(10))
    
    return words_df, textreference